# Regression tests through LDT to verify PFP API integrity

## Main Script Definitions
Below are the main definitions for functions that will then be called to execute API tests.  
This cell must be run before attempting to run a test cell.

In [71]:
import subprocess
import sys
import os
import pytest
from loguru import logger

#### Basal Functions for All Verification Tests  ######################################################
logger.remove()     # comment out for higher trace specificity than loglvl trace provides
logger.add(sys.stdout, level='INFO', format="<b><level>{level}</></> \t{message}")


## Run any generic LDT test and capture output
def run_ldt_test(ldt_command):
    logger.trace('Running run_ldt_test()...')
    # Runs LDT as a subprocess
    result = subprocess.run(ldt_command, capture_output=True, text=True, shell=True)
    return result.stdout


## Strip LDT log apart and parse as JSON to search through for keys to match on
def strip_ldt_log(log):
    logger.trace('Running strip_ldt_log()...')
    try:
        # Find the position of the last common trace line for all successful requests
        start_position = log.find("TRACE 	Running 'response_handling.handle_response'...")

        # Extract the relevant part of the output (from the common trace line to the end)
        response_json = log[start_position:].strip()
        return response_json
    
    # Error handlers
    except ValueError as json_error:
        logger.error(f"Error decoding JSON: {json_error}")
        return None

    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        return None
#######################################################################################################

## Test a generic causal pathway's functionality
def causal_pathway_test(pathway, measure, acceptable_by):
    logger.info(f'Running causal_pathway_test for {pathway}...')
    ldt_command = f"python3 ldt.py --loglvl trace --target local --CP {pathway} --respond"
    
    test_output = run_ldt_test(ldt_command)
    logger.info('LDT subprocess finished')

    # Call func to strip end of response log to use for expeced outcome matching
    output_data = strip_ldt_log(test_output)

    ## Assert expected outcome is being seen in output
    if output_data != None:
        logger.debug(output_data)
        assert f"selected_candidate.measure:	{measure}" in output_data
        assert f"selected_candidate.acceptable_by:	['{acceptable_by}']" in output_data
 
    else:
        logger.error("An error occured. Verification could not be completed.") 


    
## Test student_t_cleaner functionality
def cleaner_test():
    logger.info('Running cleaner_test...')
    ldt_command = "python3 ldt.py --loglvl trace --target local --sendLocals 1 --localPath Regression_tests/PFP/no-significant-data.json"
    
    test_output = run_ldt_test(ldt_command)
    logger.info('LDT subprocess finished')

    # Call func to strip end of response log to use for expeced outcome matching
    output_data = strip_ldt_log(test_output)

    ## Assert expected outcome is being seen in output
    if output_data != None:
        logger.debug(output_data)
        assert 'detail' in output_data
        assert '400-error' in output_data
        assert 'Status Code:	400' in output_data
 
    else:
        logger.error("An error occured. Verification could not be completed.") 


### Causal Pathway Verification
By manipulating pathway, measure, and acceptable_by arguments, we can test any of the causal pathway testing suite responses

In [73]:
# Social Worse
## Data pulled from:     https://github.com/Display-Lab/knowledge-base/blob/main/vignettes/dev_templates/causal_pathway_test_suite/social_worse_cptest.json
causal_pathway_test('social_worse', 'GLU01', 'Social Worse')

# Social Better
## Data pulled from:     https://github.com/Display-Lab/knowledge-base/blob/main/vignettes/dev_templates/causal_pathway_test_suite/social_better_cptest.json
causal_pathway_test('social_better', 'SUS04', 'Social better')

# Improving
## Data pulled from:     https://github.com/Display-Lab/knowledge-base/blob/main/vignettes/dev_templates/causal_pathway_test_suite/improving_cptest.json
causal_pathway_test('improving', 'BP03', 'Improving')



INFO 	Running causal_pathway_test for social_worse...


INFO 	LDT subprocess finished


### No Significant Data Test
Tests input message in `Regression_tests/PFP/no-significant-data.json`   
- Verifies data cleaning in `PFP/bit_stomach/student_t_cleaner.py` fully functional

In [ ]:
cleaner_test()